# OpenAI Assistants - Building Agentic RAG with the Function Calling, Retrieval, and Code Interpreter Tools

Today we'll explore using OpenAI's Python SDK to create, manage, and use the OpenAI Assistant API!

## Dependencies

We'll start, as we usually do, with some dependiencies and our API key!

In [63]:
%pip install -qU openai

In [64]:
from getpass import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key:")

## Simple Assistant

Let's create a simple Assistant to understand more about how the API works to start!

### OpenAI Client

At the core of the OpenAI Python SDK is the Client!

> NOTE: For ease of use, we'll start with the synchronous `OpenAI()`. OpenAI does provide an `AsyncOpenAI()` that you could leverage as well!

In [65]:
from openai import OpenAI

client = OpenAI()

### Creating An Assistant

Leveraging what we know about the OpenAI API from previous sessions - we're going to start by simply initializing an Assistant.

Before we begin, we need to think about a few customization options we have:

- `name` - Straight forward enough, this is what our Assistant's name will be
- `instructions` - similar to a system message, but applied at an Assistant level, this is how we can guide the Assistant's tone, behaviour, functionality, and more!
- `model` - this will allow us to choose which model we would prefer to use for our Assistant

Let's start by setting some instructions for our Assistant.



In [77]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown Fill out the fields below to add your Assistant's name, instructions, and desired model!

name = "Yohan's LLM Assistant" # @param {type: "string"}
instructions = "You are a helpful assistant." # @param {type: "string"}
model = "gpt-3.5-turbo" # @param ["gpt-3.5-turbo", "gpt-4-turbo-preview", "gpt-4"]

### Initialize Assistant

Now that we have our desired name, instruction, and model - we can initialize our Assistant!

In [78]:
assistant = client.beta.assistants.create(
    name=name,
    instructions=instructions,
    model=model,
)

Let's examine our `assistant` object and see what we find!

In [79]:
assistant

Assistant(id='asst_Y2mEPY3yLISCvvZNutDaGRjM', created_at=1713295247, description=None, file_ids=[], instructions='You are a helpful assistant.', metadata={}, model='gpt-3.5-turbo', name="Yohan's LLM Assistant", object='assistant', tools=[], top_p=1.0, temperature=1.0, response_format='auto')

There are a number of useful parameters here, but we'll call out a few:

- `id` - since we may have multiple Assistant's, knowing which Assistant we're interacting with will help us ensure the desired user experience!
- `description` - A natrual language description of our Assistant could help others understand what it's supposed to do!
- `file_ids` - if we wanted to use the Retrieval tool, this would let us know what files we had given our Assistant

### Creating a Thread

Behind the scenes our Assistant is powered by the idea of "threads".

You can think of threads as individual conversations that interact with the Assistant.

Let's create a thread now!

In [80]:
thread = client.beta.threads.create()

Let's look at our `thread` object.

In [81]:
thread

Thread(id='thread_cvAdT5xvDXpWTeFVa2fpWIvx', created_at=1713295251, metadata={}, object='thread')

Notice some key attributes:

- `id` - since each Thread is like a conversation, we need some way to specify which thread we're dealing with when interacting with them
- `tool_resources` - this will become more relevant as we add tools since we'll need a way to verify which tools we have access to when interacting with our Assistant

### Adding Messages to Our Thread

Now that we have our Thread (or conversation) we can start adding messages to it!

Let's add a simple message that asks about how our Assistant is feeling.

Notice the parameters we're leveraging:

- `thread_id` - since each Thread is like a conversation, we need some way to address a specific conversation. We can use `thread.id` to do this.
- `role` - similar to when we used our chat completions endpoint, this parameter specifies who the message is coming from. You can leverage this in the same ways you would through the chat completions endpoint.
- `content` - this is where we can place the actual text our Assistant will interact with

> NOTE: Feel free to substitute a relevant message based on the Assistant you created

In [82]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"How old are you?"
)

Again, let's examine our `message` object!

In [83]:
message

Message(id='msg_Hju9fSJ1gkqQrvzDPyZl6dTL', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='How old are you?'), type='text')], created_at=1713295255, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_cvAdT5xvDXpWTeFVa2fpWIvx')

### Running Our Thread

Now that we have an Assistant, and we've given that Assistant a Thread, and we've added a Message to that Thread - we're ready to run our Assistant!

Notice that this process lets us add (potentially) multiple messages to our Assistant. We can leverage that behaviour for few/many-shot examples, and more!

In [84]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown We can also override the Assistant's instructions when we run a thread.

# @markdown Use one of the [Prompt Principles for Instruction](https://arxiv.org/pdf/2312.16171v1.pdf) to improve the likeliehood of a correct or valuable response from your Assistant.

additional_instructions = "Where applicable, think through your response step-by-step." # @param {type: "string"}

Let's run our Thread!

In [85]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions=instructions + " " + additional_instructions
)

Now that we've run our thread, let's look at the object!

In [86]:
run

Run(id='run_SfsARgHuajU48CALK2wwqDLB', assistant_id='asst_Y2mEPY3yLISCvvZNutDaGRjM', cancelled_at=None, completed_at=None, created_at=1713295260, expires_at=1713295860, failed_at=None, file_ids=[], incomplete_details=None, instructions='You are a helpful assistant. Where applicable, think through your response step-by-step.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_cvAdT5xvDXpWTeFVa2fpWIvx', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0)

Notice we have access to a few very powerful parameters in this `run` object.

- `completed_at` - this will help us determine when we can expect to retrieve a response
- `failed_at` - this can highlight any issues our run ran into
- `status` - is another way we can understand how the flow is going

### Retrieving Our Run

Now that we've created our run, let's retrieve it.

We're going to wrap this in a simple loop to make sure we're not retrieving it too early.

In [87]:
import time

while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

In [88]:
print(run.status)

completed


Now that our run is completed - we can retieve the messages from our thread!

Notice that our run helps us understand how things are going - but it isn't where we're going to find our responses or messages. Those are added on the backend into our thread.

This leads to a simple, but important, flow:

1. We add messages to a thread.
2. We create a run on that thread.
3. We wait until the run is finished.
4. We check our thread for the new messages.

### Checking Our Thread

Now we can get a list of messages from our thread!

In [89]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [90]:
messages.data[0]

Message(id='msg_gUjWKoDVNOyfWVCMZetCv4H3', assistant_id='asst_Y2mEPY3yLISCvvZNutDaGRjM', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='I am here to assist you with any questions or tasks you may have. How can I help you today?'), type='text')], created_at=1713295261, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_SfsARgHuajU48CALK2wwqDLB', status=None, thread_id='thread_cvAdT5xvDXpWTeFVa2fpWIvx')

## Adding Tools

Now that we have an understanding of how Assistant works, we can start thinking about adding tools.

We'll go through 3 separate tools and explore how we can leverage them!

Let's start with the most familiar tool - the Retriever!


### Creating an Assistant with the Retriever Tool

The first thing we'll want to do is create an assistant with the Retriever tool.

This is also going to require some data. We'll provided data - but you're very much encouraged to use your own files to explore how the Assistant works for your use case.

#### Collect and Add Data

First, we need some data. Second, we need to add the data to our Assistant!

Let's start with grabbing some data!

In [91]:
# !wget https://www.gutenberg.org/files/84/84-h/84-h.htm -o frankenstein.html

Now we can upload our file!

Pay attention to [this](https://platform.openai.com/docs/assistants/tools/supported-files) documentation to see what kinds of files can be uploaded.

> NOTE: Per the OpenAI [docs](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval) The maximum file size is 512 MB and no more than 2,000,000 tokens (computed automatically when you attach a file)

In [92]:
file_reference = client.files.create(
  file=open("frankenstein.html", "rb"),
  purpose='assistants'
)

Let's look at what our `file_reference` contains!

In [93]:
file_reference

FileObject(id='file-OKohewH4QI0GidqqJzxrkwpj', bytes=466599, created_at=1713295298, filename='frankenstein.html', object='file', purpose='assistants', status='processed', status_details=None)

#### Create and Use Assistant

Now that we have our file - we can attach it to an Assistant, and we can give that Assistant the ability to use it for retrieval through the Retrieval tool!

> NOTE: Please pay attention to [pricing](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval) and don't forget to delete your files when you're done!

In [94]:
assistant = client.beta.assistants.create(
  name=name + " + Retrieval",
  instructions=instructions,
  model=model,
  tools=[{"type": "retrieval"}],
  file_ids=[file_reference.id]
)

Let's try submitting a message to our Assistant and seeing what kind of answer we get!

We'll outline the steps needed to do this in full:

1. Create an Assistant
2. Create a Thread
3. Add Messages to that Thread
4. Create a Run on that Thread
5. Wait for Run to Complete
6. Collect Messages from the Thread

Let's do that below!

In [95]:
# Create a Thread
thread = client.beta.threads.create()

# Add Messages to that Thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"What are the first words Victor Frankenstein speaks?"
)

# Create a Run on that Thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Wait for Run to Complete
while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  print(run.status)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

# Collect Messages from the Thread
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

queued
in_progress
in_progress


Let's look at the final result!

In [96]:
messages

SyncCursorPage[Message](data=[Message(id='msg_UnExIobL51txQTWwcCI43CWU', assistant_id='asst_o2TIJALRLRyP6gfsJaVtxfnr', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='The first words spoken by Victor Frankenstein in the book are: "It was on a dreary night of November that I beheld the accomplishment of my toils" .'), type='text')], created_at=1713295330, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_NdMUmyN5RfOnH8LizkMhQJh7', status=None, thread_id='thread_YMsQ4T9aMPvbkCnY5PaZndax'), Message(id='msg_TbLvKfY9vhAIdusKQKBiOPie', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What are the first words Victor Frankenstein speaks?'), type='text')], created_at=1713295327, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_YMsQ4T9aMPvbkC

Let's do some clean up to make sure we're not being charged anything extra by deleting our resources.

In [97]:
# file_deletion_status = client.beta.assistants.files.delete(
#   assistant_id=assistant.id,
#   file_id=file_reference.id
# )

### Creating an Assistant with the Code Interpreter Tool

Now that we've explored the Retrieval Tool - let's try the Code Interpreter tool!

The process will be almost exactly the same - but we can explore a different query, and we'll add our file at the Message level!

In [98]:
assistant = client.beta.assistants.create(
  name=name + " + Code Interpreter",
  instructions=instructions,
  model=model,
  tools=[{"type": "code_interpreter"}],
)

In the following example, we'll also see how we can package the Thread creation with the Message adding step!

> NOTE: Files added at the message/thread level will not be available to the Assistant outside of that Thread.

In [104]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "What kind of file is this? Look at both the file extension and the contents of the file to decide your answer.",
      "file_ids": [file_reference.id]
    }
  ]
)

> NOTE: Remember that we create runs at the *thread* level - and so don't need the message object to continue.

In [105]:
# Create a Run on that Thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Wait for Run to Complete
while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  print(run.status)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

# Collect Messages from the Thread
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

queued
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress


We can check the specific steps that the Code Interpreter ran to figure out what steps the Assistant took!

In [106]:
run_steps = client.beta.threads.runs.steps.list(
  thread_id=thread.id,
  run_id=run.id
)

In [107]:
for step in run_steps.data:
  print(step.step_details)

MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_bGPQA99k6iHlnr2oNqc1t24T'), type='message_creation')
ToolCallsStepDetails(tool_calls=[CodeInterpreterToolCall(id='call_i7fbMCW6u8jkhIpigCWxmV8t', code_interpreter=CodeInterpreter(input="# Read the first few lines of the file to determine its type\r\nwith open(file_path, 'r') as file:\r\n    first_few_lines = [next(file) for _ in range(5)]\r\n\r\nfirst_few_lines", outputs=[CodeInterpreterOutputLogs(logs='[\'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">\\n\',\n \'<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en"><head>\\n\',\n \'<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\\n\',\n \'<meta http-equiv="Content-Style-Type" content="text/css">\\n\',\n \'<title>The Project Gutenberg eBook of Frankenstein, by Mary Wollstonecraft Shelley</title>\\n\']', type='logs')]), type='code_interpreter')], type='tool_calls')
Mes

In [108]:
messages

SyncCursorPage[Message](data=[Message(id='msg_bGPQA99k6iHlnr2oNqc1t24T', assistant_id='asst_PsozHk33ObBUDST6BxFAuhnC', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Based on the contents of the file, it appears to be an HTML file. The presence of HTML tags such as <!DOCTYPE html> and <html> indicates that this file is structured as an HTML document.\n\nTherefore, the file is most likely an HTML file. If you need further assistance or information about this file, feel free to let me know.'), type='text')], created_at=1713295521, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_PrmnxB6H4Dm0og4fG6wGyMPN', status=None, thread_id='thread_6nXqQpW1McRt94KSnpSysPiQ'), Message(id='msg_sP4CgVZbv2AVHzEWf76zdlpS', assistant_id='asst_PsozHk33ObBUDST6BxFAuhnC', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="The file extension did not provide any information abou

In [116]:
# file_deletion_status = client.beta.assistants.files.delete(
#   assistant_id=assistant.id,
#   file_id=file_reference.id
# )

And there you go!

We've fit our Assistant with an awesome Code Interpreter that lets our Assistant run code on our provided files!

### Creating an Assistant with a Function Calling Tool

Let's finally create an Assistant that utilizes the Function Calling API.

We'll start by creating a function that we wish to be called.

We'll utilize DuckDuckGo search to allow our Assistant to have the most up to date information!

In [109]:
%pip install -qU duckduckgo_search

In [110]:
from duckduckgo_search import DDGS

def duckduckgo_search(query):
  with DDGS() as ddgs:
    results = [r for r in ddgs.text(query, max_results=5)]
    return "\n".join(result["body"] for result in results)

Let's test our function to make sure it behaves as we expect it to.

In [111]:
duckduckgo_search("Who is the current captain of the Winnipeg Jets?")

"Lowry has been a part of the Jets organization since June 25, 2011, when he was selected in the third round, 67 th overall, after putting up 37 points in 36 games with the Swift Current Broncos of ...\nAdam Lowry, who has been a Jet since 2011 when he was drafted 67th overall, is the new captain of the NHL team — its third since relocating to Winnipeg from Atlanta in 2011. Andrew Ladd served ...\nLowry will follow Andrew Ladd and Blake Wheeler to serve as the third captain of the new Winnipeg Jets franchise. - Sep 12, 2023. After a season without a captain, the Winnipeg Jets have named ...\nWinnipeg played without a captain last season after stripping Blake Wheeler of the title in September. Lowry is entering his 10th season with the Jets, who drafted him in the third round in 2011.\nThe Winnipeg Jets have officially made the decision on which player will wear the captain's 'C' for the 2023-24 season, and hopefully onward. That player is 30-year-old centre Adam Lowry."

d:\Workspaces\AIMakerspace\AI-Engineering-Cohort-2\.venv\Lib\site-packages\curl_cffi\aio.py:205: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


Now we need to express how our function works in a way that is compatible with the OpenAI Function Calling API.

We'll want to provide a `JSON` object that includes what parameters we have, how to call them, and a short natural language description.

In [113]:
ddg_function = {
    "name" : "duckduckgo_search",
    "description" : "Answer non-technical questions. ",
    "parameters" : {
        "type" : "object",
        "properties" : {
            "query" : {
                "type" : "string",
                "description" : "The search query to use. For example: 'Who is the current Goalie of the Colorado Avalance?'"
            }
        },
        "required" : ["query"]
    }
}

#### ❓QUESTION:

Why does the description key-value pair matter?

#### ANSWER

The `description` attributes are where we provide helpful prompts (or maybe prompt-like annotations) to the LLM so it understands what to do with the related node. For example, the `description` attribute below the "name: duckduckgo_search" indicates that this function can be called when answers to non-technical questions are required. Similarly, the `description` attribute inside the `query` node indicates that the query parameter is what needs to be passed to the duckduckgo_search function to conduct a browser search.

Now when we create our Assistant - we'll want to include the function description as a tool using the following format.

In [114]:
assistant = client.beta.assistants.create(
    name=name + " + Function Calling API",
    instructions=instructions,
    tools=[
        {"type": "function",
         "function" : ddg_function
        }
    ],
    model=model
)

We need to make a few modifications to our Assistant to include the ability to make calls to our local function and pass the results back to our Assistant for further generation.

In [115]:
import json

def wait_for_run_completion(thread_id, run_id):
    while True:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        print(f"Current run status: {run.status}")
        if run.status in ['completed', 'failed', 'requires_action']:
            return run

def submit_tool_outputs(thread_id, run_id, tools_to_call):
    tool_output_array = []
    for tool in tools_to_call:
        output = None
        tool_call_id = tool.id
        function_name = tool.function.name
        function_args = tool.function.arguments

        if function_name == "duckduckgo_search":
            print("Consulting Duck Duck Go...")
            output = duckduckgo_search(query=json.loads(function_args)["query"])

        if output:
            tool_output_array.append({"tool_call_id": tool_call_id, "output": output})

    print(tool_output_array)

    return client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread_id,
        run_id=run_id,
        tool_outputs=tool_output_array
    )

def print_messages_from_thread(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    for msg in messages:
        print(f"{msg.role}: {msg.content[0].text.value}")

def use_assistant(query, assistant_id, thread_id=None):
  thread = client.beta.threads.create()

  message = client.beta.threads.messages.create(
      thread_id=thread.id,
      role="user",
      content=query,
  )

  print("Creating Assistant ")

  run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
  )

  print("Querying OpenAI Assistant Thread.")

  run = wait_for_run_completion(thread.id, run.id)

  if run.status == 'requires_action':
    run = submit_tool_outputs(thread.id, run.id, run.required_action.submit_tool_outputs.tool_calls)
    run = wait_for_run_completion(thread.id, run.id)

  print_messages_from_thread(thread.id)

  return thread.id

#### ❓ QUESTION:

Outline, in simple terms, what the `use_assistant` helper function is doing.

#### ANSWER:

The `use_assistant` function invokes the LLM assistant (i.e., agent) via the following steps:
1) Creating a new thread
2) Using the provided query string to create a message on the newly created thread
3) Creating a run using the thread and the previously created assistant, and waiting for the completion of the run
4) If the intermediate response indicates that an Internet search is required (i.e., when run.status = 'requires_action'), then using the submit_tool_outputs() helper function invoke the duckduckgo_search() function with the query
5) Printing the content of the message

In [117]:
use_assistant("Who is the current Captain of the Winnipeg Jets?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: requires_action
Consulting Duck Duck Go...
[{'tool_call_id': 'call_DVUKy85BLmPvTufwSaYJThSA', 'output': "Lowry has been a part of the Jets organization since June 25, 2011, when he was selected in the third round, 67 th overall, after putting up 37 points in 36 games with the Swift Current Broncos of ...\nLowry will follow Andrew Ladd and Blake Wheeler to serve as the third captain of the new Winnipeg Jets franchise. - Sep 12, 2023. After a season without a captain, the Winnipeg Jets have named ...\nThe Winnipeg Jets have a new leader, one year after stripping the C from Blake Wheeler and deciding to play without a captain. Adam Lowry, who has been a Jet since 2011 when he was drafted 67th ...\nBy Murat Ates. Sep 13, 2023. 13. Adam Lowry sat between his coach, Rick Bowness, and the GM who drafted him, Kevin Cheveldayoff, in front of an enormous, nine-panel screen at Winnipeg's newly ...\nWinnipeg played without a

'thread_RjMT7MyPKFEeTgiNspIJ4RCj'

## Wrapping it All Together

Now we can create an Assistant with all of the available tools and see how it responds to various queries!

In [118]:
assistant = client.beta.assistants.create(
    name=name + " + All Tools",
    instructions=instructions,
    tools=[
        {"type": "code_interpreter"},
        {"type": "retrieval"},
        {"type": "function", "function" : ddg_function}
    ],
    model=model,
    file_ids=[file_reference.id],
)

In [120]:
use_assistant("Who is the current Captain of the Winnipeg Jets?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: completed
assistant: The file you uploaded is the Project Gutenberg eBook of "Frankenstein" by Mary Wollstonecraft Shelley. Unfortunately, I could not find information about the current Captain of the Winnipeg Jets in this document. Would you like me to perform a search using an external source to find the information you need?
user: Who is the current Captain of the Winnipeg Jets?


'thread_p7L0C11i46w4OecZDmp7k66e'

In [121]:
use_assistant("Who is the author of the supplied file?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: completed
assistant: The author of the supplied file titled "Frankenstein; or, the Modern Prometheus" is Mary Wollstonecraft Shelley【5†source】.
user: Who is the author of the supplied file?


'thread_TNLgnhrvowA0Z4Jg6JVICjSq'

In [122]:
use_assistant("How many bytes is the provided file?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: requires_action
Consulting Duck Duck Go...
[{'tool_call_id': 'call_GfxolgoX2OheQVB3YjyQFq8B', 'output': "1. You can use the stat() method from the os module. You can provide it with a path in the form of a string, bytes or even a PathLike object. It works with file descriptors as well. import os. res = os.stat(filename) res.st_size # this variable contains the size of the file in bytes. answered Aug 29, 2020 at 5:31.\nIt's important to get the file size in Python to monitor file size or in case of ordering files in the directory according to file size. Method 1: Using getsize function of os.path module. This function takes a file path as an argument and it returns the file size (bytes). Example: Python3\nThe os.stat() method returns the statistics of a file such as metadata of a file, creation or modification date, file size, etc. First, import the os module. Next, use the os.stat('file_path') method to get the f

'thread_dugxji6jt14OGb4VRNYRvuH7'

#### ❓ QUESTION:

Notice that our response can go through multiple paths, given that:

What is "deciding" to use the tool?

#### ANSWER:

Here the LLM is the entity taking the decision to use the tool (within an API architecture that possibly lets it return a special token setting the run.status attribute to `requires_action`). It uses the user query, along with the assistant instructions and the various system/user/assistant prompts within the context, to decide if `requires_action` needs to be set.

### Adding JSON Mode for More Agentic Behaviour

Finally, we have the ability to select tools - all we need to do now is set up a process to allow us to create some kind of loop and make decisions about whether or not the response is complete or not.

We'll leverage the OpenAI completions end-endpoint with JSON mode to let us understand when we've adequately answered our user's question!

In [123]:
completed_template = \
"""
Does this response adequately answer the user's query?

Please return your response in JSON format - with key: "completed" and either True (if completed) or False (if not completed)

User Query:
{query}

Assistant Response:
{response}
"""

def is_complete(query, response):
  completed_response = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": completed_template.format(query=query, response=response),
          }
      ],
      model=model,
      response_format={"type" : "json_object"}
  )

  return completed_response

In [124]:
query = "How many bytes is the provided file?"

thread_id_for_response = use_assistant(query, assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: completed
assistant: The provided file is 466,599 bytes in size.
user: How many bytes is the provided file?


Now we can observe JSON mode in action!

In [125]:
messages = client.beta.threads.messages.list(thread_id=thread_id_for_response)
response = messages.data[0].content[0].text.value
completed_flag = json.loads(is_complete(query, response).choices[0].message.content)

In [126]:
completed_flag

{'completed': True}

## 🚧 BONUS CHALLENGE 🚧:

Use the components we've constructed so far to build a loop that lets us continue to query the Assistant if the response is not completed!

In [61]:
### YOUR CODE HERE

# Make Sure You Delete Resources

Make sure you delete all the resources you created!

This function will help you do so!

In [127]:
file_deletion_status = client.beta.assistants.files.delete(
  assistant_id=assistant.id,
  file_id=file_reference.id
)